# Data Sim V005

* Dynamic noise, suited to do averaging in bins.

In [1]:
BROKER = "mqtt.eclipseprojects.io"
TOPIC = "gw01/duese03"

#BROKER = "kleve.cool"
#TOPIC = "gw/rolf"

USER = "pub"
PW = "pub"

STDDEV = 5.

import numpy as np
from bqplot import pyplot as plt, LinearScale
from numpy import arange, pi, sin
import ipywidgets as widgets
from ipywidgets import VBox, HBox
from IPython.display import display, clear_output
import time
import paho.mqtt.client as mqtt

In [2]:
out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout('HERE SOME MESSAGES WILL BE SHOWN!\n')
display(out)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [3]:
# Given function
def f(x, A=1, X=400, x0=0):
    y = abs(A*sin(2*pi*1/X*(x-x0)))
    y[y<0] = 0
    return y

# Data
x = arange(0, 1000.1, 5)
y = f(x, A=200, X=400)

N = len(x)

# Static moise
# x = x + np.random.normal(0,5.,N)
# y = y + np.random.normal(0,5.,N)

y0 = 400

# Create the figure and scales
fig = plt.figure(title="Function f(x)", animation_duration=0)

x_scale = LinearScale(min = -10, max = 1010)
y_scale = LinearScale(min = -10, max = 410)

# Create the profile plot
profile = plt.scatter(x, y, colors=['blue'], default_size = 20, labels=['f(x)'], scales={'x': x_scale, 'y': y_scale})
plt.xlabel("x")
plt.ylabel("y")

xs = x[0]
ys = y[0]

# Create the sample vertical line plot
beam = plt.plot([xs, xs], [y0, ys], colors=['red'], labels=['sample'], scales={'x': x_scale, 'y': y_scale})

# Create the lower end symbol (open circle)
lower_end_symbol = plt.scatter(x=[xs], y=[ys], marker='circle', colors=['red'], default_size = 50)
upper_end_symbol = plt.scatter(x=[xs], y=[y0], marker='square', colors=['red'], default_size = 100)

# sample point with noise
sample_noisy = plt.scatter(x=[xs], y=[ys], marker='circle', colors=['black'], default_size = 50)

# Create the slider
slider = widgets.IntSlider(min = 0, max = N-1, value=0, description="Choose xs", orientation="horizontal")

# MQTT setup
def on_connect(client, userdata, flags, rc):
#    print("Connected with result code " + str(rc))
    out.append_stdout("Connected with result code " + str(rc) + "\n")

client = mqtt.Client()
client.on_connect = on_connect

client.username_pw_set(USER, PW)
client.connect(BROKER, 1883, 60)
#client.subscribe(TOPIC)

client.loop_start()

# Define the update function for the slider
def update_x(change):
    xs = x[change['new']]
    ys = y[change['new']]
    beam.x = [xs, xs]
    beam.y = [y0, ys]
    lower_end_symbol.x = [xs]
    lower_end_symbol.y = [ys]
    upper_end_symbol.x = [xs]
    upper_end_symbol.y = [y0]

    xs_noisy = xs + np.random.normal(0,STDDEV)
    ys_noisy = ys + np.random.normal(0,STDDEV)

    sample_noisy.x = [xs_noisy]
    sample_noisy.y = [ys_noisy]
                      

    # Publish data over MQTT
    t = time.time()

#    data = "{},{},{}".format(t, xs, ys)
    data = "{},{},{}".format(t, xs_noisy, ys_noisy)

    client.publish(TOPIC, data)



# Link the slider to the update function
slider.observe(update_x, 'value')

# Play function for automatic slider movement
play = widgets.Play(min=0, max=len(x)-1, interval=100, description="Play", step=1, continuous_update=False)

# Update function for the play widget
def update_play(change):
    current_value = change['new']
    max_value = change['owner'].max
    min_value = change['owner'].min

    # Check if reached the limit
    if current_value >= max_value:
        change['owner'].step = -1
    elif current_value <= min_value:
        change['owner'].step = 1

# Link the play widget to the update function
play.observe(update_play, 'value')

# Join the play widget and slider using jslink
widgets.jslink((play, 'value'), (slider, 'value'))

# Arrange the slider and play button side by side
controls = HBox([slider, play])

# Display the plot and controls together using VBox
display(VBox([fig, controls]))
